# Setting up libraries

https://www.ndtv.com/business/marketdata/stocks-gainers/niftyinfrast_daily

In [7]:
import quandl
quandl.ApiConfig.api_key = 'MBMzvkxtv63KjFEV-tL6'
from quandl.errors.quandl_error import NotFoundError

In [8]:
import pandas as pd

In [9]:
slist=pd.read_csv("all_sectors.csv")

In [10]:
sl=slist["Symbol"]
sl

0           LT
1    ICICIBANK
2    EICHERMOT
3         HDFC
4        WIPRO
5    SUNPHARMA
6          ITC
7     VIDEOIND
8     HINDZINC
9          TCS
Name: Symbol, dtype: object

## Query SEtup

In [11]:
d1="2017-09-14"
d2="2018-09-14"
tick="BRITANNIA"
query="NSE/"+tick

In [12]:
df_base=quandl.get(query, start_date=d1, end_date=d2, column_index='0')
df=df_base

In [13]:
df.head()

""
Date
2017-09-14
2017-09-15
2017-09-18
2017-09-19
2017-09-20


## Fetching Data

In [14]:
for tick in sl:
    query="NSE/"+tick
    df_temp=quandl.get(query, start_date=d1, end_date=d2, column_index='5')
    df_temp.columns=[tick]
    df=pd.concat([df, df_temp], join_axes=[df.index], axis=1)
    print("Done for: {0}".format(tick))

Done for: LT
Done for: ICICIBANK
Done for: EICHERMOT
Done for: HDFC
Done for: WIPRO
Done for: SUNPHARMA
Done for: ITC
Done for: VIDEOIND
Done for: HINDZINC
Done for: TCS


In [15]:
df
# df[df["ITC"]>0]["DABUR"]
# query="NSE/MCDHOLDING"
# quandl.get(query, start_date=d1, end_date=d2, column_index='5')
df.to_csv("test.csv")

## Fetching data for NSE 50

In [16]:
nse=pd.read_csv("nse50.csv", index_col="Date", usecols=["Date","Close"])
nse.columns=['NSE']
nse.head()

,NSE
Date,
2017-09-13,NaN
2017-09-14,10086.599609
2017-09-15,10085.400391
2017-09-18,10153.099609
2017-09-19,10147.549805


In [17]:
nse=nse.drop(['2017-09-13'])
nse.head()

,NSE
Date,
2017-09-14,10086.599609
2017-09-15,10085.400391
2017-09-18,10153.099609
2017-09-19,10147.549805
2017-09-20,10141.150391


## Finding daily returns 

In [18]:
import numpy as np

In [19]:
def asset_returns(df):
    df_ret=df_base
    for col in df.columns:
        df_temp2=df[[col]].apply(lambda x: np.log(x/x.shift(1)))
        df_ret=pd.concat([df_ret, df_temp2], join_axes=[df.index], axis=1)
    return df_ret

df_ret=asset_returns(df)

In [20]:
df_ret.to_csv("Returns.csv")
df_ret.head()

,LT,ICICIBANK,EICHERMOT,HDFC,WIPRO,SUNPHARMA,ITC,VIDEOIND,HINDZINC,TCS
Date,,,,,,,,,,
2017-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-15,-0.001484,-0.007176,-0.001787,-0.001610,0.008082,-0.000667,-0.009976,-0.048267,-0.001970,0.007651
2017-09-18,0.020855,0.004790,0.023410,0.001582,0.012347,-0.006218,-0.008577,-0.005510,0.021622,-0.002797
2017-09-19,-0.009782,0.005616,-0.011114,-0.010867,-0.000173,-0.010127,0.003178,-0.013908,-0.008074,-0.000320
2017-09-20,0.008165,-0.014013,0.001220,0.008268,0.009120,-0.020171,0.013719,0.000000,-0.011578,-0.001883


In [21]:
nse_ret=asset_returns(nse)
nse_ret.head()

,NSE
Date,
2017-09-14,NaN
2017-09-15,-0.000119
2017-09-18,0.006690
2017-09-19,-0.000547
2017-09-20,-0.000631


## Asset Annual  Returns & Risk

In [22]:
sr_ret=df_ret.mean()*252

In [23]:
sr_rsk=df_ret.std()

In [26]:
df_val=pd.DataFrame(index=sr_ret.index)
df_val["Return"]=sr_ret
df_val["Risk"]=sr_rsk
df_val.to_csv("RetunRiskAsset.csv")
df_val.head()

,Return,Risk
LT,0.113909,0.013708
ICICIBANK,0.112865,0.019889
EICHERMOT,-0.088382,0.015083
HDFC,0.084166,0.012126
WIPRO,0.154502,0.012334


## Asset Beta Function

In [30]:
def beta_calc(stock, nse):
    beta=stock.cov(nse)/nse.var()
    return beta 

## Asset Beta Calculation

In [31]:
sr_beta=[]
for col in df_ret.columns:
    beta=beta_calc(df_ret[col], nse_ret["NSE"])
    sr_beta.append(beta)
print sr_beta

[1.3156966519488111, 1.5695832978429325, 0.789718678592294, 1.0971556071963802, 0.3548681486507837, 1.014476067719258, 0.849629588090378, 0.4204443812548077, 1.1842593564907737, -0.41716406482009466]


In [62]:
df_val["Beta"]=sr_beta
df_val.head()
df_val.to_csv("AssetProfile.csv")

### Importing for Permutation and combinations

In [33]:
from itertools import combinations, permutations
comb= permutations(df_ret.columns, 4)

In [34]:
comb_l=[]
for c in comb:
    comb_l.append(list(c))
len(comb_l)

5040

### Setting up weights

In [35]:
weights=(0.1,0.2,0.3,0.4)

In [36]:
df_comb= pd.DataFrame(comb_l, columns=["A","B","C","D"])
columns=df_comb.columns
df_comb

,A,B,C,D
0,LT,ICICIBANK,EICHERMOT,HDFC
1,LT,ICICIBANK,EICHERMOT,WIPRO
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA
3,LT,ICICIBANK,EICHERMOT,ITC
4,LT,ICICIBANK,EICHERMOT,VIDEOIND
5,LT,ICICIBANK,EICHERMOT,HINDZINC
6,LT,ICICIBANK,EICHERMOT,TCS
7,LT,ICICIBANK,HDFC,EICHERMOT
8,LT,ICICIBANK,HDFC,WIPRO
9,LT,ICICIBANK,HDFC,SUNPHARMA


### Return Function

In [37]:
#     comb_ret+=df_comb.loc[0,col]*wgt
def calc_ret(row_no, weights,columns, factor_name):
    comb_ret=0
    for col, wgt in zip(columns, weights):
        tick=df_comb.loc[row_no,col]
        comb_ret+=df_val.loc[tick,factor_name]*wgt
    
    return comb_ret 

## 1. Portfolio Return and Beta Calculation

In [38]:
ret_lst=[]
beta_lst=[]

for row in df_comb.index:
    ret_lst.append(calc_ret(row, weights,columns, "Return"))
    beta_lst.append(calc_ret(row, weights,columns, "Beta"))

df_comb["Return"]=ret_lst
df_comb["Beta"]=beta_lst

df_comb.head()

,A,B,C,D,Return,Beta
0,LT,ICICIBANK,EICHERMOT,HDFC,0.041116,1.121264
1,LT,ICICIBANK,EICHERMOT,WIPRO,0.069250,0.824349
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA,0.103566,1.088192
3,LT,ICICIBANK,EICHERMOT,ITC,0.056649,1.022254
4,LT,ICICIBANK,EICHERMOT,VIDEOIND,-0.608158,0.850580


### Correlation matrix calculation

In [39]:
df_corr=df_ret.corr()
df_corr

,LT,ICICIBANK,EICHERMOT,HDFC,WIPRO,SUNPHARMA,ITC,VIDEOIND,HINDZINC,TCS
LT,1.000000,0.343376,0.254578,0.345342,0.148970,0.150102,0.235392,0.128054,0.296323,-0.002227
ICICIBANK,0.343376,1.000000,0.150682,0.178685,-0.054827,0.079972,0.202889,0.080219,0.261701,-0.037936
EICHERMOT,0.254578,0.150682,1.000000,0.209263,0.081114,0.069650,0.131455,0.067445,0.194155,-0.082394
HDFC,0.345342,0.178685,0.209263,1.000000,0.069532,0.076764,0.287477,0.054906,0.197727,-0.111986
WIPRO,0.148970,-0.054827,0.081114,0.069532,1.000000,0.104565,0.064064,-0.079821,0.111830,0.035235
SUNPHARMA,0.150102,0.079972,0.069650,0.076764,0.104565,1.000000,0.097007,0.005715,0.230578,0.102813
ITC,0.235392,0.202889,0.131455,0.287477,0.064064,0.097007,1.000000,-0.005943,0.127575,0.012724
VIDEOIND,0.128054,0.080219,0.067445,0.054906,-0.079821,0.005715,-0.005943,1.000000,0.135344,-0.002841
HINDZINC,0.296323,0.261701,0.194155,0.197727,0.111830,0.230578,0.127575,0.135344,1.000000,-0.044236
TCS,-0.002227,-0.037936,-0.082394,-0.111986,0.035235,0.102813,0.012724,-0.002841,-0.044236,1.000000


In [40]:
from matplotlib import pyplot
import seaborn as sns; sns.set()
# uniform_data = np.random.rand(10, 12)
pyplot.figure(figsize=(16, 16))

ax = sns.heatmap(df_corr, cmap="Greens", vmin=0.01, vmax=.5, square=True )

In [43]:
df_corr.to_csv("CorrelationMatrix.csv")


### Risk Function

In [44]:
import math

def calc_rsk(row, weights):
    asset_list=list(df_comb.loc[row,:"D"])
    asset_weight={}
    
    
#     This coomand willl help us to form key value pair of weights of portfolio
    for a, w in zip(asset_list, weights):
        asset_weight[a]=w 
        
        
        
    comb_risk=0
#   (weights x )^2
    for a in asset_list: 
        comb_risk+=(df_val["Risk"][a] * asset_weight[a])**2
    
            
    for p in combinations(asset_list, 2):
        a1=p[0]
        a2=p[1]
        comb_risk += 2*asset_weight[a1]*df_val["Risk"][a1] \
                      *asset_weight[a2]*df_val["Risk"][a2] \
                      * df_corr[a1][a2]
        
    comb_risk=math.sqrt(comb_risk)
    
    return comb_risk

## 2. Portfolio Risk Calculation

In [45]:
rsk_lst=[]
for row in df_comb.index:
    rsk_lst.append(calc_rsk(row,weights))
df_comb["Risk"]=rsk_lst
df_comb.head()

,A,B,C,D,Return,Beta,Risk
0,LT,ICICIBANK,EICHERMOT,HDFC,0.041116,1.121264,0.009732
1,LT,ICICIBANK,EICHERMOT,WIPRO,0.069250,0.824349,0.008850
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA,0.103566,1.088192,0.011270
3,LT,ICICIBANK,EICHERMOT,ITC,0.056649,1.022254,0.009445
4,LT,ICICIBANK,EICHERMOT,VIDEOIND,-0.608158,0.850580,0.016326


## 3. Sharpe Ratio

In [46]:
df_comb["SharpeRatio"]=(df_comb["Return"]-0.069)/df_comb["Risk"]
df_comb.head()

,A,B,C,D,Return,Beta,Risk,SharpeRatio
0,LT,ICICIBANK,EICHERMOT,HDFC,0.041116,1.121264,0.009732,-2.865310
1,LT,ICICIBANK,EICHERMOT,WIPRO,0.069250,0.824349,0.008850,0.028276
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA,0.103566,1.088192,0.011270,3.066987
3,LT,ICICIBANK,EICHERMOT,ITC,0.056649,1.022254,0.009445,-1.307751
4,LT,ICICIBANK,EICHERMOT,VIDEOIND,-0.608158,0.850580,0.016326,-41.476397


## 4.Tryernor Ratio

In [47]:
df_comb["TreynorRatio"]=(df_comb["Return"]-0.069)/df_comb["Beta"]
df_comb.head()

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio
0,LT,ICICIBANK,EICHERMOT,HDFC,0.041116,1.121264,0.009732,-2.865310,-0.024869
1,LT,ICICIBANK,EICHERMOT,WIPRO,0.069250,0.824349,0.008850,0.028276,0.000304
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA,0.103566,1.088192,0.011270,3.066987,0.031765
3,LT,ICICIBANK,EICHERMOT,ITC,0.056649,1.022254,0.009445,-1.307751,-0.012082
4,LT,ICICIBANK,EICHERMOT,VIDEOIND,-0.608158,0.850580,0.016326,-41.476397,-0.796113


## 5. Information Ratio

In [48]:
nse_ret_annual=nse_ret["NSE"].mean()*252
nse_ret_annual
print("Annnual Return of NSE:{}".format(nse_ret_annual) )
df_comb["InformationRatio"]=(df_comb["Return"]-nse_ret_annual)/df_comb["Risk"]
df_comb.head()

Annnual Return of NSE:0.143751879301


,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio
0,LT,ICICIBANK,EICHERMOT,HDFC,0.041116,1.121264,0.009732,-2.865310,-0.024869,-10.546593
1,LT,ICICIBANK,EICHERMOT,WIPRO,0.069250,0.824349,0.008850,0.028276,0.000304,-8.418000
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA,0.103566,1.088192,0.011270,3.066987,0.031765,-3.565578
3,LT,ICICIBANK,EICHERMOT,ITC,0.056649,1.022254,0.009445,-1.307751,-0.012082,-9.222502
4,LT,ICICIBANK,EICHERMOT,VIDEOIND,-0.608158,0.850580,0.016326,-41.476397,-0.796113,-46.055003


## 6. Sortino Ratio

### 6.1 Finding downside risk.

In [49]:
def down_risk(row, df_comb):
    comb=df_comb.loc[row, :"D"]
    MARR=.1
    ret=0
    for asset, w in zip(comb, weights):
        ret+=df_ret[asset]*w
    port_ret=pd.DataFrame(ret)
    port_ret.columns=["PortRet"]
    port_ret["PortRet"]=port_ret["PortRet"]-MARR/len(port_ret)
    port_ret[port_ret < 0] = 0
    return port_ret.std()
    

In [50]:
down_risk_lst=[]
for index in range(0,len(df_comb)):
    down_risk_lst.append(down_risk(index,df_comb)[0])

down_risk_lst

[0.005538812851363892,
 0.005370309526071841,
 0.00679522927446056,
 0.005872736844023781,
 0.008721789851651833,
 0.006459707302189797,
 0.005434738772944077,
 0.005775038718717983,
 0.004908137158248969,
 0.006482046648091514,
 0.005770538180280856,
 0.008620238961289477,
 0.006293805914322786,
 0.0050013353447700545,
 0.005612276873343902,
 0.004955667219769828,
 0.006421900288235641,
 0.005448545819486048,
 0.008455488347245152,
 0.005991476410243952,
 0.005468640604059972,
 0.006388645434861636,
 0.005905886883838867,
 0.00581967923313535,
 0.006190093924036207,
 0.009167308417581059,
 0.006732672563448496,
 0.005929224265734946,
 0.005993222651905401,
 0.005671944130308916,
 0.005321280296619958,
 0.0066595319693054725,
 0.008499669741746898,
 0.006451550230224306,
 0.005197563650269592,
 0.007577239039417519,
 0.007248059176644221,
 0.006994433845677488,
 0.008294372628478524,
 0.007180620832540122,
 0.008096579102116228,
 0.006778202336333676,
 0.006355910773964136,
 0.00603307

In [51]:
df_comb["DownRisk"]=down_risk_lst

### 6.2 Finding sortino Ratio

In [52]:
df_comb["SortioRatio"]=(df_comb["Return"]-0.069)/df_comb["DownRisk"]
df_comb.head()

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio
0,LT,ICICIBANK,EICHERMOT,HDFC,0.041116,1.121264,0.009732,-2.865310,-0.024869,-10.546593,0.005539,-5.034347
1,LT,ICICIBANK,EICHERMOT,WIPRO,0.069250,0.824349,0.008850,0.028276,0.000304,-8.418000,0.005370,0.046599
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA,0.103566,1.088192,0.011270,3.066987,0.031765,-3.565578,0.006795,5.086845
3,LT,ICICIBANK,EICHERMOT,ITC,0.056649,1.022254,0.009445,-1.307751,-0.012082,-9.222502,0.005873,-2.103145
4,LT,ICICIBANK,EICHERMOT,VIDEOIND,-0.608158,0.850580,0.016326,-41.476397,-0.796113,-46.055003,0.008722,-77.639761


### Efficient = Return/Risk

In [53]:
df_comb["Efficient"]=df_comb["Return"]/df_comb["Risk"]
df_comb.head()

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio,Efficient
0,LT,ICICIBANK,EICHERMOT,HDFC,0.041116,1.121264,0.009732,-2.865310,-0.024869,-10.546593,0.005539,-5.034347,4.224928
1,LT,ICICIBANK,EICHERMOT,WIPRO,0.069250,0.824349,0.008850,0.028276,0.000304,-8.418000,0.005370,0.046599,7.824643
2,LT,ICICIBANK,EICHERMOT,SUNPHARMA,0.103566,1.088192,0.011270,3.066987,0.031765,-3.565578,0.006795,5.086845,9.189201
3,LT,ICICIBANK,EICHERMOT,ITC,0.056649,1.022254,0.009445,-1.307751,-0.012082,-9.222502,0.005873,-2.103145,5.997990
4,LT,ICICIBANK,EICHERMOT,VIDEOIND,-0.608158,0.850580,0.016326,-41.476397,-0.796113,-46.055003,0.008722,-77.639761,-37.250097


In [54]:
df_comb.to_csv("AllInfoPortfolio.csv")

# Top 2 Returns

In [55]:
df_comb.nlargest(n=2,columns="Return")

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio,Efficient
304,LT,ITC,WIPRO,SUNPHARMA,0.178458,0.813746,0.010078,10.860732,0.134511,3.443648,0.006308,17.352257,17.707100
808,ICICIBANK,ITC,WIPRO,SUNPHARMA,0.178354,0.839135,0.010048,10.882654,0.130317,3.443500,0.006191,17.663038,17.749393


# Top 2 Efficient

In [56]:
df_comb.nlargest(n=2,columns="Efficient")

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio,Efficient
759,ICICIBANK,SUNPHARMA,ITC,WIPRO,0.158045,0.756690,0.008295,10.734710,0.117677,1.723119,0.005106,17.439547,19.052892
255,LT,SUNPHARMA,ITC,WIPRO,0.158150,0.731301,0.008344,10.684443,0.121906,1.725559,0.005175,17.225880,18.953975


# Top 2 Sharpe Ratio

In [57]:
df_comb.nlargest(n=2,columns="SharpeRatio")

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio,Efficient
815,ICICIBANK,ITC,SUNPHARMA,WIPRO,0.169775,0.773174,0.009085,11.092325,0.130339,2.864343,0.005538,18.195302,18.687194
311,LT,ITC,SUNPHARMA,WIPRO,0.169879,0.747786,0.009150,11.024708,0.134904,2.855349,0.005656,17.835608,18.565465


# Top 2 Treynor Ratio 

In [58]:
df_comb.nlargest(n=2,columns="TreynorRatio")

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio,Efficient
311,LT,ITC,SUNPHARMA,WIPRO,0.169879,0.747786,0.009150,11.024708,0.134904,2.855349,0.005656,17.835608,18.565465
1823,HDFC,ITC,SUNPHARMA,WIPRO,0.166905,0.725932,0.008995,10.884011,0.134868,2.573890,0.005536,17.684872,18.554699


# Top 2 Information Ratio

In [59]:
df_comb.nlargest(n=2,columns="InformationRatio")

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio,Efficient
304,LT,ITC,WIPRO,SUNPHARMA,0.178458,0.813746,0.010078,10.860732,0.134511,3.443648,0.006308,17.352257,17.707100
808,ICICIBANK,ITC,WIPRO,SUNPHARMA,0.178354,0.839135,0.010048,10.882654,0.130317,3.443500,0.006191,17.663038,17.749393


# Top 2 Sortino Ratio

In [60]:
df_comb.nlargest(n=2,columns="SortioRatio")

,A,B,C,D,Return,Beta,Risk,SharpeRatio,TreynorRatio,InformationRatio,DownRisk,SortioRatio,Efficient
815,ICICIBANK,ITC,SUNPHARMA,WIPRO,0.169775,0.773174,0.009085,11.092325,0.130339,2.864343,0.005538,18.195302,18.687194
311,LT,ITC,SUNPHARMA,WIPRO,0.169879,0.747786,0.009150,11.024708,0.134904,2.855349,0.005656,17.835608,18.565465


## Saving Work

In [61]:
df_comb.to_csv("AllPortfolio.csv")